In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from tqdm.notebook import tqdm
import re
import os
from tqdm import tqdm

In [2]:
processed_dir = "/data1/normantm/angel/sequencing/original_single_cell_24wells_output"

In [4]:
filenames = sorted([entry for entry in os.listdir(processed_dir) if entry.startswith('new_processed') and entry.endswith('.csv')])

In [8]:
identity = []
reporter = []
for f in tqdm(filenames):
    transcript, sample = f.rstrip(".csv").split("_")[-2:]
    df = pd.read_csv(f"{processed_dir}/{f}")
    df["sample"] = sample
    if transcript == "identity":
        identity.append(df)
    elif transcript == "reporter":
        reporter.append(df)
    else:
        print("transcript should be either identity or reporter")

100%|████████████████████████████████████████████| 4/4 [08:20<00:00, 125.06s/it]


In [10]:
identity_df = pd.concat(identity, ignore_index=True)
reporter_df = pd.concat(reporter, ignore_index=True)

In [12]:
identity_df = identity_df[["p1_identity", "p2_identity", "barcode_mapped", "well", "UMI", "sample"]].value_counts().to_frame("n")
reporter_df = reporter_df[["p1_identity", "p2_identity", "barcode_mapped", "well", "UMI", "sample"]].value_counts().to_frame("n")

In [15]:
identity_UMI = identity_df.groupby(["p1_identity", "p2_identity", "barcode_mapped", "well", "sample"]).size().to_frame("UMI")
reporter_UMI = reporter_df.groupby(["p1_identity", "p2_identity", "barcode_mapped", "well", "sample"]).size().to_frame("UMI")

In [21]:
UMI = identity_UMI.merge(reporter_UMI, left_index=True, right_index=True, suffixes=("_identity", "_reporter"), how="outer").fillna(0)

In [26]:
UMI.to_csv("/data1/normantm/angel/Pscreen_data/single_cell_screen/UMI_counts.csv")